In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [20]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))[:100]):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(p)+list(a))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(p)+list(a))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-100.csv")
df = df.iloc[:100]
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/rb_exact_50-100.csv")

 20%|███████████████▌                                                              | 20/100 [40:01<2:39:57, 119.97s/it]

## Heuristic - Round

In [12]:
from src.heuristic import naive_round

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))[:100]):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-100.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:01<00:00,  2.42s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   1.595451e+04          0.312121      2.395702
std    1.544777e+05          2.200007      2.172108
min    3.598118e+02          0.000000      0.920691
25%    4.483671e+02          0.000000      1.246543
50%    5.084547e+02          0.000000      1.796755
75%    5.518894e+02          0.000000      2.808077
max    1.545284e+06         16.557013     18.618939
Number of infeasible solution: 2


## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:46<00:00,  2.46it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.749761e+12               0.0      0.403884
std    9.192329e+12               0.0      0.166594
min    5.558174e+02               0.0      0.245428
25%    7.700089e+02               0.0      0.310472
50%    8.632258e+02               0.0      0.353868
75%    9.996323e+02               0.0      0.414785
max    4.999519e+13               0.0      1.257725
Number of infeasible solution: 0


In [8]:
df = pd.read_csv("result/rb_heur_n1_50-100.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.999690e+12               0.0      0.409453
std    9.845793e+12               0.0      0.148936
min    5.645990e+02               0.0      0.245428
25%    7.619726e+02               0.0      0.325797
50%    8.919459e+02               0.0      0.358961
75%    1.025256e+03               0.0      0.422262
max    4.999237e+13               0.0      1.028245
Number of infeasible solution: 0
Number of None values:  0


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 20996.05
Epoch 1, Validation Loss: 4016.29
Epoch 2, Validation Loss: 1759.47
Epoch 3, Validation Loss: 1213.61
Epoch 4, Validation Loss: 1522.16
Epoch 5, Validation Loss: 1312.35
Epoch 6, Validation Loss: 1165.38
Epoch 7, Validation Loss: 1032.75
Epoch 8, Validation Loss: 1098.03
Epoch 9, Validation Loss: 1492.84
Epoch 10, Validation Loss: 1189.45
Epoch 11, Validation Loss: 831.75
Epoch 12, Validation Loss: 845.00
Epoch 13, Validation Loss: 885.85
Epoch 14, Validation Loss: 662.87
Epoch 15, Validation Loss: 820.98
Epoch 16, Validation Loss: 756.71
Epoch 17, Validation Loss: 650.82
Epoch 18, Validation Loss: 665.11
Epoch 19, Validation Loss: 884.84
Epoch 20, Validation Loss: 730.10
Epoch 21, Validation Loss: 614.67
Epoch 22, Validation Loss: 644.56
Epoch 23, Validation Loss: 720.34
Epoch 24, Validation Loss: 749.18
Epoch 25, Validation Loss: 666.92
Epoch 26, Validation Loss: 606.64
Epoch 27, Validation Loss: 566.92
Epoch 28, Validation Loss: 766.88
Epoch 29, Va

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 53.46it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    568.468975          0.005120      0.003273
std     205.694313          0.101694      0.000969
min     266.924016          0.000000      0.000999
25%     366.757142          0.000000      0.002518
50%     558.681739          0.000000      0.003003
75%     720.295802          0.000000      0.003999
max    1574.572316          2.409778      0.008513
Number of infeasible solution: 3


In [10]:
df = pd.read_csv("result/rb_lr_50-100.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean    548.524412          0.006933      0.003540
std     231.017347          0.069333      0.001223
min     276.415199          0.000000      0.001999
25%     351.154360          0.000000      0.002767
50%     516.172182          0.000000      0.003098
75%     714.485029          0.000000      0.004256
max    1419.896277          0.693326      0.008513
Number of infeasible solution: 1
Number of None values:  0


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 18587.63
Epoch 1, Validation Loss: 3448.91
Epoch 2, Validation Loss: 2870.52
Epoch 3, Validation Loss: 1232.80
Epoch 4, Validation Loss: 1303.39
Epoch 5, Validation Loss: 1380.01
Epoch 6, Validation Loss: 1107.19
Epoch 7, Validation Loss: 14572.58
Epoch 8, Validation Loss: 2064.73
Epoch 9, Validation Loss: 24670.45
Epoch 10, Validation Loss: 843.25
Epoch 11, Validation Loss: 727.15
Epoch 12, Validation Loss: 2243217268.91
Epoch 13, Validation Loss: 794.01
Epoch 14, Validation Loss: 827.54
Epoch 15, Validation Loss: 809.01
Epoch 16, Validation Loss: 1533.53
Epoch 17, Validation Loss: 712.35
Epoch 18, Validation Loss: 736.32
Epoch 19, Validation Loss: 808.03
Epoch 20, Validation Loss: 725.17
Epoch 21, Validation Loss: 896.27
Epoch 22, Validation Loss: 731.22
Epoch 23, Validation Loss: 639.29
Epoch 24, Validation Loss: 747.64
Epoch 25, Validation Loss: 799.46
Epoch 26, Validation Loss: 965.93
Epoch 27, Validation Loss: 714.63
Epoch 28, Validation Loss: 673.89
Epo

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.03it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    639.164363          0.011845      0.002797
std     138.688235          0.374560      0.000703
min     417.931821          0.000000      0.000999
25%     545.062545          0.000000      0.002113
50%     608.273523          0.000000      0.002994
75%     689.761520          0.000000      0.003030
max    1598.102928         11.844629      0.006236
Number of infeasible solution: 1


In [11]:
df = pd.read_csv("result/rb_lt_50-100.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())

           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean    647.550143          0.118446      0.002797
std     159.574255          1.184463      0.000684
min     421.836907          0.000000      0.001509
25%     559.713165          0.000000      0.002455
50%     610.942312          0.000000      0.002830
75%     689.762984          0.000000      0.003013
max    1598.102928         11.844629      0.005658
Number of infeasible solution: 1
Number of None values:  0


## Parametric Learning Then Rounding

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [17]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [18]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 18669.99
Epoch 1, Validation Loss: 1604.38
Epoch 2, Validation Loss: 852.39
Epoch 3, Validation Loss: 494.47
Epoch 4, Validation Loss: 672.31
Epoch 5, Validation Loss: 318.87
Epoch 6, Validation Loss: 519.18
Epoch 7, Validation Loss: 336.93
Epoch 8, Validation Loss: 293.51
Epoch 9, Validation Loss: 293.15
Epoch 10, Validation Loss: 608.17
Epoch 11, Validation Loss: 306.33
Epoch 12, Validation Loss: 444.61
Epoch 13, Validation Loss: 492.69
Epoch 14, Validation Loss: 269.47
Epoch 15, Validation Loss: 363.40
Epoch 16, Validation Loss: 267.53
Epoch 17, Validation Loss: 517.68
Epoch 18, Validation Loss: 285.02
Epoch 19, Validation Loss: 282.13
Epoch 20, Validation Loss: 244.92
Epoch 21, Validation Loss: 286.68
Epoch 22, Validation Loss: 256.09
Epoch 23, Validation Loss: 278.70
Epoch 24, Validation Loss: 254.02
Epoch 25, Validation Loss: 241.33
Epoch 26, Validation Loss: 254.99
Epoch 27, Validation Loss: 230.28
Epoch 28, Validation Loss: 279.49
Epoch 29, Validation 

In [19]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))[:100]):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-100.csv")
df = df.iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.65it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000        100.000000    100.000000
mean    603.639254          0.095145      0.000709
std     328.650344          0.951451      0.001140
min     195.644007          0.000000      0.000000
25%     304.208432          0.000000      0.000000
50%     495.564522          0.000000      0.001000
75%     842.349442          0.000000      0.001000
max    1399.521807          9.514506      0.010791
Number of infeasible solution: 1
